In [2]:
# %pip install -qq torchviz
# %pip install graphviz
%pip install -qq -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "vinai/PhoGPT-4B-Chat"
base_model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/home/hungson175/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/f56fc6d71f147a3a293fdab56676337dc6f641e1/configuration_mpt.py:114: UserWarning: alibi or rope is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi or rope is turned on, setting `learned_pos_emb` to `False.`')
/home/hungson175/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/f56fc6d71f147a3a293fdab56676337dc6f641e1/configuration_mpt.py:141: UserWarning: If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".
  warnings.warn(UserWarning('If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".'))


In [8]:
print(base_model.device)

cpu


In [4]:
print(base_model)
# Access the embedding layer
embedding_layer = base_model.transformer.wte
print("Embedding")
print(embedding_layer)

# Access the first transformer block
first_transformer_block = base_model.transformer.blocks[0]
print("First Transformer Block")
print(vars(first_transformer_block))

# # Access the self-attention layer within the first transformer block
# self_attention_layer = first_transformer_block.attn
# print("Self-Attention Layer")
# print(self_attention_layer)

MPTForCausalLM(
  (transformer): MPTModel(
    (wte): SharedEmbedding(20480, 3072)
    (emb_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-31): 32 x MPTBlock(
        (norm_1): LPLayerNorm((3072,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (Wqkv): Linear(in_features=3072, out_features=9216, bias=True)
          (out_proj): Linear(in_features=3072, out_features=3072, bias=True)
        )
        (norm_2): LPLayerNorm((3072,), eps=1e-05, elementwise_affine=True)
        (ffn): MPTMLP(
          (up_proj): Linear(in_features=3072, out_features=12288, bias=True)
          (down_proj): Linear(in_features=12288, out_features=3072, bias=True)
        )
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_ffn_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm_f): LPLayerNorm((3072,), eps=1e-05, elementwise_affine=True)
  )
)
Embedding
SharedEmbedding(20480, 3072)
First Transformer Block
{'traini

In [4]:
def print_model_parameters(model, num_only=False):
    if num_only:
        print(f"Number of parameters: {model.num_parameters()}")
        return
    for name, param in model.named_parameters():
        print(f"Layer: {name} | Size: {param.size()} | Trainable: {param.requires_grad}")

In [5]:
import torch
from torchviz import make_dot

def model_graph(model):
    # Dummy input to pass through the model
    dummy_input = torch.ones(1, 10).long()

    # Pass the dummy input through the model
    output = model(dummy_input)

    # Visualize the model
    dot = make_dot(output.logits, params=dict(model.named_parameters()))
    dot.render("huy/model_architecture", format="svg")

In [11]:
def infer(model, tokenizer, instruction):
    PROMPT_TEMPLATE = f"### Câu hỏi: {instruction}\n### Trả lời:"  

    input_prompt = PROMPT_TEMPLATE.format_map({"instruction": instruction})  

    input_ids = tokenizer(input_prompt, return_tensors="pt")  

    outputs = model.generate(  
        inputs=input_ids["input_ids"].to("cuda"),  
        attention_mask=input_ids["attention_mask"].to("cuda"),  
        do_sample=True,  
        temperature=1.0,  
        top_k=50,  
        top_p=0.9,  
        max_new_tokens=1024,  
        eos_token_id=tokenizer.eos_token_id,  
        pad_token_id=tokenizer.pad_token_id  
    )  

    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]  
    response = response.split("### Trả lời:")[1]
    return response

In [12]:
def infer_model(model, tokenizer, input):
    response = infer(model, tokenizer, input)
    return response

In [13]:
import copy
import gc

instruction = "Viết bài văn nghị luận xã hội về việc đi học"
print_model_parameters(base_model, num_only=True)

Number of parameters: 3688077312


In [15]:
# Single layer reduction
# for i, _ in enumerate(base_model.transformer.blocks):
for i in range(22,32):
    # Copy model and remove layers
    new_model = copy.deepcopy(base_model)
    del new_model.transformer.blocks[i]
    print_model_parameters(new_model, num_only=True)
    new_model.to("cuda")
    new_model.eval()

    # Infer
    res = infer_model(new_model, tokenizer, instruction)
    print(f'Response remove layer #{i}:')
    print(res)
    print('---------------------------------------------------')
    
    # Delete model
    new_model.to("cpu")
    del new_model
    gc.collect()
    torch.cuda.empty_cache()


Number of parameters: 3574791168


/home/hungson175/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/f56fc6d71f147a3a293fdab56676337dc6f641e1/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/memory usage. Consider integrating ' + 'into attn_bias once and passing that to each attention ' + 'module instead.')


Response remove layer #22:
Việc đi học là một trong những điều cơ bản và quan trọng nhất của đời người. Ở mọi độ tuổi, mỗi người cần đi học để tiếp thu tri thức và kỹ năng cần thiết trong cuộc sống. Có nhiều độ tuổi đi học nhưng phổ biến nhất là học sinh sinh viên vì đối tượng này cần trang bị cho mình những kiến thức cơ bản, những kỹ năng sống cần thiết và phát triển tư duy sáng tạo. Học là quá trình trau dồi tri thức, rèn kỹ năng và đạo đức, bồi đắp tri thức cần thiết. Từ xưa đến nay, học luôn là một đề tài được cả xã hội quan tâm. Từ những độ tuổi khác nhau, việc học được tiếp thu và thực hiện ở khắp các quốc gia trên thế giới. Đi học là việc đầu tiên để tiếp nhận tri thức, văn hóa của quốc gia. Từ khi bắt đầu cất tiếng khóc chào đời, ai cũng đều đến trường và khi lớn lên lại quay lại trường để hoàn thành nhiệm vụ học tập của mình. Điều này diễn ra ở khắp mọi nơi trên thế giới và là một trong những nhiệm vụ quan trọng và cơ bản nhất trong cuộc đời của một con người.

Có rất nhiều cá

In [14]:
# Single layer reduction
# for i, _ in enumerate(base_model.transformer.blocks):
for i in range(5,8):
    new_model = copy.deepcopy(base_model)
    del new_model.transformer.blocks[i*4:i*4+4]
    print_model_parameters(new_model, num_only=True)
    new_model.to("cuda")
    new_model.eval()
    res = infer_model(new_model, tokenizer, instruction)
    print(f'Response remove layers #{i*4}-{i*4+3}:')
    print(res)
    print('---------------------------------------------------')
    new_model.to("cpu")
    del new_model
    gc.collect()
    torch.cuda.empty_cache()

Number of parameters: 3234932736


/home/hungson175/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/f56fc6d71f147a3a293fdab56676337dc6f641e1/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/memory usage. Consider integrating ' + 'into attn_bias once and passing that to each attention ' + 'module instead.')


Response remove layers #20-23:
Đi học. Từ lâu, con người có nhu cầu vật chất và tinh thần cần có những thứ cần thiết, từ thời trang cho con người, vật chất cho con người mà người dân lao động không có thời gian và không biết cách sử dụng. Trong xã hội, nhu cầu con người phát triển và đã đưa vào cho sản xuất, thì lúc đó đất nước phải chuẩn bị, còn lại thì những người già và những em nhỏ chưa được có nên việc của đất nước rất quan trọng là cần có lớp đi học là cho con em. Vì người già và các em thì không phải là lớp cũng như những nhà doanh nghiệp sản xuất kinh doanh về lương thực, thực phẩm, dược phẩm... đều phải được đào tạo. Lớp để cho con em có thể đào tạo để phát triển con người của đất nước.
Đi học, là con người sống và học tập, có được sự thành công, đó là quá trình vận hành, học từ những cái cụ thể đến những điều mà con người cần biết. Đi học, rất quan trọng cho cuộc sống, để giúp cho con người trở lại, với những kiến thức cần có, để vận hành cho sản xuất và cuộc sống. Có con ngư

In [9]:
new_model.to("cpu")
del new_model
gc.collect()
torch.cuda.empty_cache()

In [8]:
# Some instruction examples
# instruction = "Viết bài văn nghị luận xã hội về {topic}"
# instruction = "Viết bản mô tả công việc cho vị trí {job_title}"
# instruction = "Sửa lỗi chính tả:\n{sentence_or_paragraph}"
# instruction = "Dựa vào văn bản sau đây:\n{text}\nHãy trả lời câu hỏi: {question}"
# instruction = "Tóm tắt văn bản:\n{text}"
# instruction = "Sửa lỗi chính tả:\nTriệt phá băng nhóm kướp ô tô, sử dụng \"vũ khí nóng\""

instruction = "Viết bài văn nghị luận xã hội về việc đi học"

In [9]:
responseA = infer(model, tokenizer, instruction)

/home/hungson175/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/116013fa63f8c4025739487e1cbff65b7375bbe2/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/memory usage. Consider integrating ' + 'into attn_bias once and passing that to each attention ' + 'module instead.')


In [12]:
responseB = infer(new_model, tokenizer, instruction)

In [13]:
print(responseA)
print("-------------------------------------------------")
print(responseB)

Trong cuộc sống mỗi con người, việc học luôn là vấn đề được đặt lên hàng đầu. Học để nâng cao trình độ hiểu biết, để tự khẳng định mình, học để trở thành người có ích cho xã hội. Bởi thế, việc đi học dường như là nhiệm vụ là nghĩa vụ mà bất cứ ai cũng cần phải làm.

Đi học có thể hiểu là việc đến trường học tập của con người, học sinh đến trường để được thầy cô dạy dỗ, chỉ bảo. Đi học là một hình thức để chúng ta có thể tiếp thu kiến thức mới mẻ của nhân loại. Bởi khi chúng ta đi học, chúng ta sẽ được giáo viên cung cấp kiến thức về văn hóa, về tự nhiên, về các môn học khác trong trường. Việc học có rất nhiều cách như học qua sách giáo khoa, học qua các phương tiện thông tin đại chúng như học trên sách vở, học qua bạn bè, học qua mạng internet... và đặc biệt nhất là học qua thực tiễn. Học qua thực tiễn là quá trình chúng ta trực tiếp tiếp xúc với các kiến thức, hiểu biết các lí thuyết mà mình đang học trong cuộc sống hàng ngày.

Việc đi học không chỉ mang lại cho con người kiến thức mà

In [7]:
import evaluate
import datasets
import numpy as np
from evaluate import logging
from torch.nn import CrossEntropyLoss
import torch

class Perplexity(evaluate.Metric):
    def _info(self):
        return evaluate.MetricInfo(
            module_type="metric",
            description="_DESCRIPTION",
            citation="_CITATION",
            inputs_description="_KWARGS_DESCRIPTION",
            features=datasets.Features(
                {
                    "predictions": datasets.Value("string"),
                }
            ),
            reference_urls=["https://huggingface.co/docs/transformers/perplexity"],
        )

    def _compute(
        self, predictions, model, tokenizer, device, batch_size: int = 16, add_start_token: bool = True, max_length=None
    ):



        # if batch_size > 1 (which generally leads to padding being required), and
        # if there is not an already assigned pad_token, assign an existing
        # special token to also be the padding token
        if tokenizer.pad_token is None and batch_size > 1:
            existing_special_tokens = list(tokenizer.special_tokens_map_extended.values())
            # check that the model already has at least one special token defined
            assert (
                len(existing_special_tokens) > 0
            ), "If batch_size > 1, model must have at least one special token to use for padding. Please use a different model or set batch_size=1."
            # assign one of the special tokens to also be the pad token
            tokenizer.add_special_tokens({"pad_token": existing_special_tokens[0]})

        if add_start_token and max_length:
            # leave room for <BOS> token to be added:
            assert (
                tokenizer.bos_token is not None
            ), "Input model must already have a BOS token if using add_start_token=True. Please use a different model, or set add_start_token=False"
            max_tokenized_len = max_length - 1
        else:
            max_tokenized_len = max_length

        encodings = tokenizer(
            predictions,
            add_special_tokens=False,
            padding=True,
            truncation=True if max_tokenized_len else False,
            max_length=max_tokenized_len,
            return_tensors="pt",
            return_attention_mask=True,
        ).to(device)

        encoded_texts = encodings["input_ids"]
        attn_masks = encodings["attention_mask"]

        # check that each input is long enough:
        if add_start_token:
            assert torch.all(torch.ge(attn_masks.sum(1), 1)), "Each input text must be at least one token long."
        else:
            assert torch.all(
                torch.ge(attn_masks.sum(1), 2)
            ), "When add_start_token=False, each input text must be at least two tokens long. Run with add_start_token=True if inputting strings of only one token, and remove all empty input strings."

        ppls = []
        loss_fct = CrossEntropyLoss(reduction="none")

        for start_index in logging.tqdm(range(0, len(encoded_texts), batch_size)):
            end_index = min(start_index + batch_size, len(encoded_texts))
            encoded_batch = encoded_texts[start_index:end_index]
            attn_mask = attn_masks[start_index:end_index]

            if add_start_token:
                bos_tokens_tensor = torch.tensor([[tokenizer.bos_token_id]] * encoded_batch.size(dim=0)).to(device)
                encoded_batch = torch.cat([bos_tokens_tensor, encoded_batch], dim=1)
                attn_mask = torch.cat(
                    [torch.ones(bos_tokens_tensor.size(), dtype=torch.int64).to(device), attn_mask], dim=1
                )

            labels = encoded_batch

            with torch.no_grad():
                out_logits = model(encoded_batch, attention_mask=attn_mask).logits

            shift_logits = out_logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_attention_mask_batch = attn_mask[..., 1:].contiguous()

            perplexity_batch = torch.exp(
                (loss_fct(shift_logits.transpose(1, 2), shift_labels) * shift_attention_mask_batch).sum(1)
                / shift_attention_mask_batch.sum(1)
            )

            ppls += perplexity_batch.tolist()

        return {"perplexities": ppls, "mean_perplexity": np.mean(ppls)}

In [13]:
import copy, gc

perplexity = Perplexity()
input_texts = ["Viết bài văn nghị luận xã hội về việc đi học",
               "Write an essay on the social discourse of going to school",]

# Calculate perplexity for the base model
base_model.to("cuda")
base_model.eval()
results = perplexity.compute(model=base_model, tokenizer=tokenizer, device="cuda", batch_size=16,
                        add_start_token=False,
                        predictions=input_texts)
print("Base model:", )
print(results["perplexities"])
print(results["mean_perplexity"])


base_model.to("cpu") # ~ 14GB in mem 
gc.collect()
torch.cuda.empty_cache()

  0%|          | 0/1 [00:00<?, ?it/s]

Base model:
[8.871155738830566, 29.371994018554688]
19.121574878692627


In [12]:


removed_layers = [0,4,8,12,16,20,24,28]
num_layer = 4
for i in removed_layers:
    new_model = copy.deepcopy(base_model)
    del new_model.transformer.blocks[i:i+num_layer]
    # print_model_parameters(new_model, num_only=True)
    new_model.to("cuda")
    new_model.eval()
    
    results = perplexity.compute(model=new_model, tokenizer=tokenizer, device="cuda", batch_size=16,
                             add_start_token=False,
                             predictions=input_texts)
    print(f"Layers from {i} to {i+num_layer-1} removed---------------------------------")
    print(results["perplexities"])
    print(results["mean_perplexity"])
    
    new_model.to("cpu")
    del new_model
    gc.collect()
    torch.cuda.empty_cache()




  0%|          | 0/1 [00:00<?, ?it/s]

Layers from 0 to 3 removed---------------------------------
[5769.95458984375, 2646995.25]
1326382.6022949219


  0%|          | 0/1 [00:00<?, ?it/s]

Layers from 4 to 7 removed---------------------------------
[32.29954528808594, 161.35711669921875]
96.82833099365234


  0%|          | 0/1 [00:00<?, ?it/s]

...

Layers from 20 to 23 removed---------------------------------
[10.884724617004395, 60.91717529296875]
35.90094995498657


  0%|          | 0/1 [00:00<?, ?it/s]

Layers from 24 to 27 removed---------------------------------
[12.62756061553955, 43.02472686767578]
27.826143741607666


  0%|          | 0/1 [00:00<?, ?it/s]

Layers from 28 to 31 removed---------------------------------
[15.908745765686035, 227.23753356933594]
121.57313966751099


In [21]:
import gc 
gc.collect()
torch.cuda.empty_cache()